In [1]:
#had a lot of issues trying to install the "imputeLCMD" package, so this was how I did it
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install(c("impute", "pcaMethods"))

# install.packages("imputeLCMD")

In [2]:
setwd("/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Input")
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Output')
cur_date = "020922"

library(missForest)
library(imputeLCMD)
library(readxl)
library(tidyverse)

#reading in files
ozone_data = data.frame(read_excel("Ozone_Sterol_Asthma_data_011822.xlsx", sheet = 2)) 

Loading required package: randomForest

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: foreach

Loading required package: itertools

Loading required package: iterators

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:Matrix’:

    which


The following object is masked from ‘package:rand

Goal is to generate (impute) values for missing data. Given that this dataset contains various categories (ie. cell differentials, cytokines) a different method will be employed for each.

> Lung Function Data: Random Forest (RF)

> Soluble Mediators (everything else): Quantile Regression Imputation of Left-Censored Data (QRILC)

**Also make note of subjects/ variables with missing data.

In [3]:
# Need to impute missing data, but the distribution needs to be normalized first
ozone_data = ozone_data %>%
    mutate(Value = ifelse(Value == "NA", NA, as.numeric(Value))) %>%
    #removing all the changes and percent data, since to prevent the imputations from generating values that don't
    # make sense (ie. percents > 100); based on the raw values anyways
    filter(grepl("_Change", Variable) == FALSE & grepl("_Percent", Variable) == FALSE) #%>%
    #ended up pseudo log transforming the data to prevent infinite (NaN) values
    #mutate(Value_log = log(Value)) 

head(ozone_data)

Warning message in ifelse(Value == "NA", NA, as.numeric(Value)):
“NAs introduced by coercion”


,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,FEV1,4.39
2,4,1,AS_1,AS,P,Pre,NR,NR,Lung_Function,FEV1,4.60
3,4,1,AS_1,AS,S,Post,NR,NR,Lung_Function,FEV1,4.39
4,4,1,AS_1,AS,S,Pre,NR,NR,Lung_Function,FEV1,NA
5,7,2,AS_2,AS,P,Post,R,NR,Lung_Function,FEV1,2.96
6,7,2,AS_2,AS,P,Pre,R,NR,Lung_Function,FEV1,3.08


In [4]:
#now splitting dataset based on category, sample, time point
preimputed_df = ozone_data %>%
    group_by(Category, Sample_Type, Time_Point) %>%
    group_split()

cell_differential_plasma_post_df = preimputed_df[[1]]
cell_differential_plasma_pre_df = preimputed_df[[2]]
cell_differential_sputum_post_df = preimputed_df[[3]]
cell_differential_sputum_pre_df = preimputed_df[[4]]
cytokine_plasma_post_df = preimputed_df[[5]]
cytokine_plasma_pre_df = preimputed_df[[6]]
cytokine_sputum_post_df = preimputed_df[[7]]
cytokine_sputum_pre_df = preimputed_df[[8]]
lung_function_plasma_post_df = preimputed_df[[9]]
lung_function_plasma_pre_df = preimputed_df[[10]]
lung_function_sputum_post_df = preimputed_df[[11]]
lung_function_sputum_pre_df = preimputed_df[[12]]
sterol_plasma_post_df = preimputed_df[[13]]
sterol_plasma_pre_df = preimputed_df[[14]]
sterol_sputum_post_df = preimputed_df[[15]]
sterol_sputum_pre_df = preimputed_df[[16]]

# Random Forest Imputation

In [5]:
head(lung_function_plasma_post_df)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,FEV1,4.39
7,2,AS_2,AS,P,Post,R,NR,Lung_Function,FEV1,2.96
11,3,AS_3,AS,P,Post,NR,R,Lung_Function,FEV1,4.24
13,4,AS_4,AS,P,Post,R,NR,Lung_Function,FEV1,4.99
15,5,AS_5,AS,P,Post,R,NR,Lung_Function,FEV1,2.93
16,6,AS_6,AS,P,Post,NR,NR,Lung_Function,FEV1,2.83


In [8]:
#imputing lung function data using RF
random_forest_imputation = function(dataset){
    wider_dataset = dataset %>%
        pivot_wider(names_from = Variable, values_from = Value)
    imputed_RF_object = missForest(as.matrix(wider_dataset[,10:12]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(wider_dataset[,1:9], imputed_RF_df))
    
    #ADD BACK IN OTHER METRICS
    
    return(imputed_final_df)
}

#calling fn
imputed_lung_function_plasma_post_df = random_forest_imputation(lung_function_plasma_post_df)
imputed_lung_function_plasma_pre_df = random_forest_imputation(lung_function_plasma_pre_df)
imputed_lung_function_sputum_post_df = random_forest_imputation(lung_function_sputum_post_df)
imputed_lung_function_sputum_pre_df = random_forest_imputation(lung_function_sputum_pre_df)

head(imputed_lung_function_plasma_post_df)

  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!


,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,FEV1,FVC1,FEV1.FVC1
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,4.39,4.41,99.55
2,7,2,AS_2,AS,P,Post,R,NR,Lung_Function,2.96,3.17,93.38
3,11,3,AS_3,AS,P,Post,NR,R,Lung_Function,4.24,4.37,97.03
4,13,4,AS_4,AS,P,Post,R,NR,Lung_Function,4.99,5.26,94.87
5,15,5,AS_5,AS,P,Post,R,NR,Lung_Function,2.93,3.07,95.44
6,16,6,AS_6,AS,P,Post,NR,NR,Lung_Function,2.83,2.86,98.95


# QRILC Imputation

In [9]:
head(cell_differential_plasma_post_df)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,817
7,2,AS_2,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,7992
11,3,AS_3,AS,P,Post,NR,R,Cell_Differential,Sample_Weight,2791
13,4,AS_4,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,3530
15,5,AS_5,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,3830
16,6,AS_6,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,3287


In [22]:
#imputing lung function data using RF
QRILC_imputation = function(dataset, number_of_variables){
    wider_dataset = dataset %>%
        pivot_wider(names_from = Variable, values_from = Value)
    imputed_QRILC_object = impute.QRILC(as.matrix(wider_dataset[,10:number_of_variables]))
    QRILC_df = data.frame(imputed_QRILC_object[1]) 
    #PIVOT LONGER!!!
    imputed_dataset = data.frame(cbind(dataset[,1:8], QRILC_df))
    
    return(imputed_dataset)
}

#calling fn
imputed_cell_differential_plasma_post_df = QRILC_imputation(cell_differential_plasma_post_df, 24)
imputed_cell_differential_plasma_pre_df = QRILC_imputation(cell_differential_plasma_pre_df, 24)
imputed_cell_differential_sputum_post_df = QRILC_imputation(cell_differential_sputum_post_df, 24)
imputed_cell_differential_sputum_pre_df = QRILC_imputation(cell_differential_sputum_pre_df, 24)
#imputed_cytokine_plasma_post_df = QRILC_imputation(cytokine_plasma_post_df, 40)
#imputed_cytokine_plasma_pre_df = QRILC_imputation(cytokine_plasma_pre_df, 40)
imputed_cytokine_sputum_post_df = QRILC_imputation(cytokine_sputum_post_df, 40)
imputed_cytokine_sputum_pre_df = QRILC_imputation(cytokine_sputum_pre_df, 40)
imputed_sterol_plasma_post_df = QRILC_imputation(sterol_plasma_post_df, 27)
imputed_sterol_plasma_pre_df = QRILC_imputation(sterol_plasma_pre_df, 27)
#imputed_sterol_sputum_post_df = QRILC_imputation(sterol_sputum_post_df, 27)
#imputed_sterol_sputum_pre_df = QRILC_imputation(sterol_sputum_pre_df, 27)

In [23]:
head(imputed_cell_differential_plasma_post_df)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Sample_Weight,TCC,⋯,Macrophages,Macrophages_Weight,Eosinophils,Eosinophils_Weight,Lymphocytes,Lymphocytes_Weight,BEC,BEC_Weight,SEC,Viability
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,817,395000,⋯,296250,363,0.0,0,0,0,0.0,0,371300,2.50
2,7,2,AS_2,AS,P,Post,R,NR,7992,2630000,⋯,1090924,137,88368.0,11,70747,9,88368.0,11,284040,92.40
3,11,3,AS_3,AS,P,Post,NR,R,2791,2510000,⋯,534881,191,82328.0,29,41164,15,370225.0,132,978900,44.91
4,13,4,AS_4,AS,P,Post,R,NR,3530,3420000,⋯,1539684,436,307800.0,87,8208,2,8208.0,2,533520,56.40
5,15,5,AS_5,AS,P,Post,R,NR,3830,8930000,⋯,871568,228,0.0,0,19646,5,19646.0,5,696540,85.90
6,16,6,AS_6,AS,P,Post,NR,NR,3287,405000,⋯,299619,91,1498.5,0,4536,1,6034.5,2,187110,43.20
